In [1]:
import copy
import torch
import time
import os
import random 
import numpy as np
import aux_funcs  as af
import network_architectures as arcs

from architectures.CNNs.VGG import VGG

from train_networks import *

In [2]:
random_seed = af.get_random_seed()
af.set_random_seeds()
print('Random Seed: {}'.format(random_seed))
device = af.get_pytorch_device()
models_path = 'networks/{}'.format(af.get_random_seed())
af.create_path(models_path)
af.set_logger('outputs/train_models'.format(af.get_random_seed()))

Random Seed: 1221
Using PyTorch version: 1.12.1+cu116 CUDA: True
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Creating resnet56 untrained cifar10 models...
Saving CNN...


In [19]:
tasks = ['cifar10']

cnns = [arcs.create_resnet56(models_path, 'cifar10', save_type='c')]
train(models_path, cnns, sdn=False, device=device)

KeyError: 'init_rpf_pannel'

In [13]:
cnn_model = arcs.load_model(models_path, "cifar10_resnet56_rpf_cnn", -1)

In [3]:
sdn_model = arcs.load_model(models_path, "cifar10_resnet56_rpf_sdn_sdn_training", -1)

In [4]:
import torchattacks
import torch
import torch.nn.functional as F

def evaluate_sdn_attack(model, test_loader, atk):
    atk.set_normalization_used(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    test_acc = 0
    n = 0
    model.eval()
    for i, (X, y) in enumerate(test_loader):
        X, y = X.to('cuda'), y.to('cuda')

        if model.use_rpf:
          model.random_rp_matrix()

        X_adv = atk(X, y)  # advtorch

        if model.use_rpf:
          model.random_rp_matrix()

        with torch.no_grad():
            output = model(X_adv)
        test_acc += (output.max(1)[1] == y).sum().item()
        n += y.size(0)

    pgd_acc = test_acc / n
    return pgd_acc

In [5]:
cifar = af.get_dataset("cifar10")

In [25]:
sdn_model[0].early_stop = True
sdn_model[0].need_info = False
sdn_model[0].confidence_threshold = 2
sdn_model[0].use_rpf = True

In [7]:
sdn_model[0].cuda()

ResNet_SDN(
  (init_conv): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (layers): ModuleList(
    (0): BasicBlockWOutput(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Sequential()
        (2): ReLU()
      )
    )
    (1): BasicBlockWOutput(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(16, 16

In [8]:
atk = torchattacks.FGSM(sdn_model[0], eps=8/255)

In [27]:
evaluate_sdn_attack(sdn_model[0], cifar.test_loader, atk)

0.6715

In [20]:
np.mean(sdn_res)

0.57167

In [17]:
cnn_model[0].cuda()
atk = torchattacks.FGSM(cnn_model[0], eps=8/255)
cnn_res = []
for i in range(50):
    acc = evaluate_sdn_attack(cnn_model[0], cifar.test_loader, atk)
    cnn_res.append(acc)

In [18]:
np.mean(cnn_res)

0.480906

In [30]:
import model_funcs as mf

In [31]:
mf.cnn_test(sdn_model[0], cifar.test_loader, device)

(87.15, 99.43)

In [32]:
mf.cnn_test(cnn_model[0], cifar.test_loader, device)

(86.799995, 99.43)